In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from shapely.geometry import MultiPolygon,Polygon
import shapely
from src.PlotFuncitons import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # does not affect other users
# or consider setting your dpi in rcParams to a reasonably high value (also important for exporting if not explicitly given)

# Cooler Blog zu Mapping
Folgendes ist aus dem Artikel:
https://juanitorduz.github.io/germany_plots/

Sein Skript geht noch weiter, allerdings ist das für uns nicht relevant. Falls in den späteren Zellen was nicht funktioniert wie erwartet: Alle Zellen ab hier nochmal laufen lassen. Viele Operationen sind mehr oder weniger offensichtlich inplace.

In [ ]:
# Make sure you read postal codes as strings, otherwise 
# the postal code 01110 will be parsed as the number 1110. 
plz_shape_df = gpd.read_file('Datasets/PLZ/plz-5stellig/plz-5stellig.shp', dtype={'plz': str}) # ,encoding='Windows-1252')
plz_shape_df.drop(['einwohner','qkm'],axis=1,inplace=True)
plz_shape_df.head()

In [ ]:
plt.rcParams['figure.figsize'] = [16, 11]

# Get lat and lng of Germany's main cities. 
top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Cologne': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt am Main': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'Munich': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nuremberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052)
}

fig, ax = plt.subplots()

plz_shape_df.plot(ax=ax, color='orange', alpha=0.8)

# Plot cities. 
for c in top_cities.keys():
    # Plot city name.
    ax.text(
        x=top_cities[c][0], 
        # Add small shift to avoid overlap with point.
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )
    # Plot city location centroid.
    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='Germany', 
    aspect=1.4, 
    facecolor='lightblue'
)

In [ ]:
# Create feature.
plz_shape_df = plz_shape_df.assign(first_dig_plz = lambda x: x['plz'].str.slice(start=0, stop=1))

fig, ax = plt.subplots()

plz_shape_df.plot(
    ax=ax, 
    column='first_dig_plz', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'First Digit', 'loc':'lower right'},
    cmap='tab20',
    alpha=0.9
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='Germany First-Digit-Postal Codes Areas', 
    aspect=1.4,
    facecolor='white'
)

In [ ]:
plz_region_df = pd.read_csv(
    'Datasets/PLZ/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)

plz_region_df.drop(['osm_id','ags','landkreis'], axis=1, inplace=True)

plz_region_df.head()

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)

germany_df.drop(['note'], axis=1, inplace=True)

germany_df.head()

In [ ]:
fig, ax = plt.subplots()

germany_df.plot(
    ax=ax, 
    column='bundesland', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'Bundesland', 'bbox_to_anchor': (1.35, 0.8)},
    cmap='tab20',
    alpha=0.9
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='Germany - Bundesländer', 
    aspect=1.4, 
    facecolor='white'
)

In [ ]:
plz_einwohner_df = pd.read_csv(
    'Datasets/PLZ/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)
plz_einwohner_df.drop(['note','qkm','lat','lon'],axis=1,inplace=True)
plz_einwohner_df.head()

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=germany_df, 
    right=plz_einwohner_df, 
    on='plz',
    how='left'
)

germany_df.head()

In [ ]:
fig, ax = plt.subplots()

germany_df.plot(
    ax=ax, 
    column='einwohner', 
    categorical=False, 
    legend=True, 
    cmap='RdYlGn_r',
    alpha=0.8
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Number of Inhabitants per Postal Code', 
    aspect=1.4, 
    facecolor='lightblue'
)

In [ ]:
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
germany_df['einwohner_kategorie'] = pd.cut(germany_df['einwohner'], bins=bins, labels=labels, right=False)

fig, ax = plt.subplots()

germany_df.plot(
    ax=ax, 
    column='einwohner_kategorie', 
    legend=True, 
    cmap='RdYlGn_r',
    alpha=0.8
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Number of Inhabitants per Postal Code', 
    aspect=1.4, 
    facecolor='lightblue'
)

# Ab hier kommt wieder meine Code Ursuppe

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/plz-5stellig/plz-5stellig.shp', dtype={'plz': str})
plz_map = pd.read_csv('Datasets/PLZ/georef-germany-postleitzahl.csv',sep=';',dtype={'Postleitzahl / Post code':str})
plz_map = plz_map.rename(columns={'Postleitzahl / Post code':'plz'})
merged_df = pd.merge(geo_df, plz_map, on='plz',how='left')
merged_df.head()

In [ ]:
grouped_df = merged_df.groupby('PLZ Name (short)').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()

In [ ]:
fig, ax = plt.subplots()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')

bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']

grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap='RdYlGn_r')
ax.set(
    title='Germany: Number of Inhabitants per Postal Code', 
    aspect=1.3, 
    facecolor='lightblue'
)

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
geo_df.head()

In [ ]:
fig,ax = plt.subplots()
geo_df.plot(
    ax=ax, 
    column='lan_code', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'First Digit', 'loc':'lower right'},
    cmap='tab20',
    alpha=.7
)
# Plot cities. 
for c in top_cities.keys():
    # Plot city name.
    ax.text(
        x=top_cities[c][0], 
        # Add small shift to avoid overlap with point.
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )
    # Plot city location centroid.
    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='Germany', 
    aspect=1.4, 
    facecolor='lightblue',
)

In [ ]:
inhabitans_df = pd.read_csv('Datasets/PLZ/plz_einwohner.csv',dtype={'plz':str}).rename(columns={'plz':'plz_code'})
merged_df = pd.merge(geo_df,inhabitans_df,on='plz_code',how='left')
merged_df.head()


In [ ]:
grouped_df = merged_df.groupby('krs_code').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

fig, ax = plt.subplots()
grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap='RdYlGn_r')
# Plot cities. 
for c in top_cities.keys():
    # Plot city name.
    ax.text(
        x=top_cities[c][0], 
        # Add small shift to avoid overlap with point.
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )
    # Plot city location centroid.
    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title="Germany: Number of Inhabitants per 'Kreis' Code", 
    aspect=1.4, 
    facecolor='lightblue',
)

In [ ]:
grouped_df = merged_df.groupby('plz_name').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

fig, ax = plt.subplots()
grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap='RdYlGn_r')
# Plot cities. 
for c in top_cities.keys():
    # Plot city name.
    ax.text(
        x=top_cities[c][0], 
        # Add small shift to avoid overlap with point.
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )
    # Plot city location centroid.
    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title="Germany: Number of Inhabitants per Postal Code Name", 
    aspect=1.4, 
    facecolor='lightblue',
)

In [ ]:
def load_LKS01_2019_2022(fpath:str):
    df = pd.read_excel(fpath,skiprows=3,thousands='.',decimal=',')
    df = df.rename(columns={
        'erfasste Fälle':'Anzahl erfasste Fälle', # 2019
        'erfasste Fälle davon:\nVersuche':'erfasste Fälle: Anzahl Versuche',
        'von Spalte 3\nVersuche':'erfasste Fälle: Anzahl Versuche', # 2019
        'Unnamed: 6':'erfasste Fälle: Versuche in %',
        'Tatortverteilung':'Tatortverteilung: bis unter 20.000 Einwohner',
        'Unnamed: 8':'Tatortverteilung: 20.000 bis unter 100.000',
        'Unnamed: 9':'Tatortverteilung: 100.000 bis unter 500.000',
        'Unnamed: 10': 'Tatortverteilung: 500.000 und mehr',
        'Unnamed: 11':'Tatortverteilung: unbekannt',
        'mit Schusswaffe':'mit Schusswaffe: gedroht',
        'Unnamed: 13':'mit Schusswaffe: geschossen',
        'Aufklärung':'Aufklärung: Anzahl Fälle',
        'Unnamed: 15':'Aufklärung: in % (AQ)',
        'Tatverdächtige':'Tatverdächtige: insgesamt',
        'Unnamed: 17':'Tatverdächtige: männlich',
        'von Spalte 16':'Tatverdächtige: männlich',
        'Unnamed: 18':'Tatverdächtige: weiblich',
        'Nichtdeutsche Tatverdächtige':'Nichtdeutsche Tatverdächtige: Anzahl',
        'Unnamed: 19':'Nichtdeutsche Tatverdächtige: Anzahl', # 2019
        'Unnamed: 20':'Nichtdeutsche Tatverdächtige: Anteil an TV insg. in %'
    })
    return df.drop(range(4)).reset_index(drop=True)

def load_LKS01_2015_2018(fpath):
    # confirmed for 2018,2017,2016,2015
    df = pd.read_excel(fpath,skiprows=4,thousands='.',decimal=',')
    df = df.drop(['BL-Schl.','Sort'], axis=1, errors='ignore')
    df = df.rename(columns={
        'erfasste Fälle':'Anzahl erfasste Fälle',
        'von Spalte 4 Versuche':'erfasste Fälle: Anzahl Versuche',
        'Unnamed: 6':'erfasste Fälle: Versuche in %',
        'Unnamed: 7':'erfasste Fälle: Versuche in %', # 2018
        'Aufklärung':'Aufklärung: Anzahl Fälle',
        'Unnamed: 8':'Aufklärung: in % (AQ)', # really the same?
        'Unnamed: 9':'Aufklärung: in % (AQ)', # really the same?
        'Tatver-dächtige insg.':'Tatverdächtige: insgesamt',
        'Nichtdeutsche Tat-verdächtige':'Nichtdeutsche Tatverdächtige: Anzahl',
        'Unnamed: 11':'Nichtdeutsche Tatverdächtige: Anteil an TV insg. in %',
        'Unnamed: 12':'Nichtdeutsche Tatverdächtige: Anteil an TV insg. in %' # 2018
    })
    return df.drop(range(2)).reset_index(drop=True)

def load_LKS01_2014(fpath:str='Datasets/PKS/2014/tb01_FaelleGrundtabelleLaender_excel.xlsx'):
    df = pd.read_excel(fpath,skiprows=7,thousands='.',decimal=',')
    df = df.rename(columns={
        'Strft. Schl.':'Schlüssel',
        'erfasste Fälle 2014':'Anzahl erfasste Fälle',
        'Versuche absolut':'erfasste Fälle: Anzahl Versuche',
        'Versuche in %':'erfasste Fälle: Versuche in %',
        'aufgeklärte Fälle':'Aufklärung: Anzahl Fälle',
        'AQ \nin %':'Aufklärung: in % (AQ)',
        'TV insges.':'Tatverdächtige: insgesamt',
        'NDTV insges.':'Nichtdeutsche Tatverdächtige: Anzahl',
        'NDTV in %':'Nichtdeutsche Tatverdächtige: Anteil an TV insg. in %'
    })
    return df

def load_LKS01_2013(fpath:str='Datasets/PKS/2013/tb01_FaelleGrundtabelleLaender_excel.xls'):
    df = pd.read_excel(fpath,skiprows=8,thousands='.',decimal=',')
    df = df.rename(columns={
        'Strft. Schl.':'Schlüssel',
        'erfasste Fälle 2013':'Anzahl erfasste Fälle',
        'Versuche absolut':'erfasste Fälle: Anzahl Versuche',
        'Versuche in %':'erfasste Fälle: Versuche in %',
        'aufgeklärte Fälle':'Aufklärung: Anzahl Fälle',
        'AQ \nin %':'Aufklärung: in % (AQ)',
        'TV insges.':'Tatverdächtige: insgesamt',
        'NDTV insges.':'Nichtdeutsche Tatverdächtige: Anzahl',
        'NDTV in %':'Nichtdeutsche Tatverdächtige: Anteil an TV insg. in %'
    })
    return df

class LKS01_loader(Dataset):
    def __init__(self,root_dir:str='Datasets/PKS/'):
        self.root_dir = root_dir

    def __len__(self):
        return len(os.listdir(self.root_dir))
    
    def __getitem__(self,year):
        ypath = os.path.join(self.root_dir,str(year))
        for file in os.listdir(ypath):
            fpath = os.path.join(ypath,file)
            # load table for all years
            if 2013 > year or year > 2022:
                raise IndexError(f'No data for requested year: {year}.\nNote: There is no official data before 2013 for this table.')
            if any(desi in file for desi in ['LA','Laender']):
                if 2019 <= year <= 2022:
                    return load_LKS01_2019_2022(fpath)
                if 2015 <= year <= 2018:
                    return load_LKS01_2015_2018(fpath)
                if year == 2014:
                    return load_LKS01_2014(fpath)
                if year == 2013:
                    return load_LKS01_2013(fpath)

## Reduce GeoData to Federal States

Problems:
- merging GeoDataFrame on federal states leaves noisy borders (solve by shrinking and expanding with buffer; only slight loss of precision)
- federal states in this dataset are not free of overlaps and show up in plots (solve by manually checking maps and removing overlap from the state that is too large)

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/georef-germany-postleitzahl/georef-germany-postleitzahl.shp').rename(columns={'lan_name':'Bundesland'})
bu_geo_full = geo_df.groupby('Bundesland').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x])
}).reset_index()


# reduce geo information to outline of states
bu_geo_reduced = bu_geo_full.copy()
bu_geo_reduced['geometry'] = bu_geo_full['geometry'].apply(lambda x: shapely.ops.unary_union(x))

# re-compute shapes with slight buffering to avoid holes where borders don't line up perfectly
bu_geo_smooth = bu_geo_reduced.copy()
bu_geo_smooth['geometry'] = bu_geo_reduced['geometry'].apply(
    lambda x: x.buffer(1e-5, 1, join_style=shapely.geometry.JOIN_STYLE.mitre).buffer(-1e-4, 1, join_style=shapely.geometry.JOIN_STYLE.mitre))

bu_geo_full = gpd.GeoDataFrame(bu_geo_full,geometry='geometry')
bu_geo_reduced = gpd.GeoDataFrame(bu_geo_reduced,geometry='geometry')
bu_geo_smooth = gpd.GeoDataFrame(bu_geo_smooth,geometry='geometry')

overlaps = {'BU_1':[],'BU_2':[],'geometry':[]}
for _,bu1 in bu_geo_smooth.iterrows(): 
    for _,bu2 in bu_geo_smooth.iterrows():
        if bu1['Bundesland'] != bu2['Bundesland']: # problem: this only filter self-intersection, but every other intersection is considered twice
            overlaps['BU_1'].append(bu1['Bundesland'])
            overlaps['BU_2'].append(bu2['Bundesland'])
            overlaps['geometry'].append(bu1.geometry.intersection(bu2.geometry))
overlaps = gpd.GeoDataFrame(overlaps,geometry='geometry')
overlaps = overlaps[~overlaps['geometry'].is_empty].reset_index(drop=True) # drop empty intersections

fig,axs = plt.subplots(1,3)
bu_geo_full.plot(ax=axs[0],alpha=.5,edgecolor='black')
bu_geo_reduced.plot(ax=axs[1],alpha=.5,edgecolor='black')
bu_geo_smooth.plot(ax=axs[2],alpha=.5,edgecolor='black')
overlaps.plot(ax=axs[2],facecolor='red',edgecolor='none')
for ax in axs.flatten():
    ax.set(aspect=1.4)
plt.show()

In [ ]:
geo = bu_geo_smooth.copy()

def remove_intersection(region1, region2):
    '''Removes the intersecting region from region 1

    Input:
    :param: region1: The region which is too large
    :param: region2: The region which the area actally belongs to
    
    Output:
    :return: Reduced shape of region 1'''

    reduced = []
    for pol1 in region1:
        for pol2 in region2:
            if pol1.intersects(pol2):
                # If they intersect, create a new polygon that is
                # essentially pol minus the intersection
                nonoverlap = (pol1.symmetric_difference(pol2)).difference(pol2)
                reduced.append(list(nonoverlap.geoms) if isinstance(nonoverlap,MultiPolygon) else nonoverlap)
            else:
                # Otherwise, just keep the initial polygon as it is.
                reduced.append(pol1)
                
    return shapely.ops.unary_union(reduced)

def cleanBU_(df,bu1,bu2):
    df.loc[df.loc[:,"Bundesland"] == bu1,"geometry"] = remove_intersection(
        df.loc[df.loc[:,"Bundesland"] == bu1,"geometry"],
        df.loc[df.loc[:,"Bundesland"] == bu2,"geometry"])

cleanBU_(geo,"Hessen","Niedersachsen")
cleanBU_(geo,"Hessen","Rheinland-Pfalz")
cleanBU_(geo,"Hessen","Nordrhein-Westfalen")
cleanBU_(geo,"Bayern","Baden-Württemberg")
cleanBU_(geo,"Baden-Württemberg","Hessen")
cleanBU_(geo,"Thüringen","Sachsen")
cleanBU_(geo,"Schleswig-Holstein","Hamburg")
cleanBU_(geo,"Mecklenburg-Vorpommern","Niedersachsen")
cleanBU_(geo,"Mecklenburg-Vorpommern","Brandenburg")
cleanBU_(geo,"Sachsen-Anhalt","Brandenburg")

geo.plot(alpha=0.5,aspect=1.3,edgecolor='black')
plt.show()

In [ ]:
df = LKS01_loader()[2022]
cases = df.loc[df['Schlüssel'] == '------']
merged_df = pd.merge(cases,geo,on='Bundesland')
merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [ ]:
cmap = 'RdYlGn_r'

fig, ax = plt.subplots()
merged_df.plot(ax=ax,column='Anzahl erfasste Fälle', legend=True,cmap=cmap,legend_kwds={'label':'Absolute number of crimes'})
plot_cities(ax)
ax.set(
    title="Germany 2022: Crimes per federal state", 
    aspect=1.4, 
    facecolor='lightblue',
)
plt.show()

## Cybercrime per federal state over time

In [ ]:
# This might take ~20 seconds
data = LKS01_loader()
merged_data = []
# no "%-Anteil an allen Fällen" column available before 2019; compute it manually:
print('Loading 2013 - 2022 ...')
for year in range(2013,2019):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    df_year_all = df_year[df_year['Schlüssel'] == '------']['Anzahl erfasste Fälle'].to_numpy()
    df_year = df_year.loc[df_year['Schlüssel'] == '897000']
    df_year['%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    merged_year = pd.merge(df_year,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))
for year in range(2019,2023):
    print(year)
    df_year = data[year]
    df_year = df_year.loc[df_year['Schlüssel'] == '897000']
    merged_year = pd.merge(df_year,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))

In [ ]:
vmin = min([min(x.loc[:,'%-Anteil an allen Fällen']) for x in merged_data])
vmax = max([max(x.loc[:,'%-Anteil an allen Fällen']) for x in merged_data])

fig, axs = plt.subplots(3,4,layout='constrained',figsize=(8,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        merged_data[i].plot(ax=ax,column='%-Anteil an allen Fällen', legend=False, cmap=cmap,vmin=vmin,vmax=vmax)
        ax.set(aspect=1.4,title=i+2013)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,cmap,'% of all crime')
fig.suptitle("Germany: Cybercrime per federal state (relative)")
plt.show()

Huh? Shouldn't 2020 and 2021 be overall higher than other years? (see temporal analysis)

Problem: For some reason the fraction of all crime in 2020 and 2021 is not computed wrt. the number of crimes in each federal state, but in all of Germany.

In [ ]:
def add_fraction(df_year):
    df_year_all = df_year[df_year['Schlüssel'] == '------']['Anzahl erfasste Fälle'].to_numpy()
    df_year = df_year.loc[df_year['Schlüssel'] == '897000'].copy()
    if '%-Anteil an allen Fällen' in df_year.columns:
        df_year.loc[:,'%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    else:
       df_year['%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    return pd.merge(df_year,geo,on='Bundesland')

data = LKS01_loader()
merged_data = []
# no "%-Anteil an allen Fällen" column available before 2019; compute it manually and fix 2020,2021:
print('Loading 2013 - 2022 ...')
for year in range(2013,2023):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    if year in [2019,2022]:
        df_year = df_year.loc[df_year['Schlüssel'] == '897000']
        merged_year = pd.merge(df_year,geo,on='Bundesland')
        merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))
    else:
        merged_year = add_fraction(df_year)
        merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))

In [ ]:
vmin = min([min(x.loc[:,'%-Anteil an allen Fällen']) for x in merged_data])
vmax = max([max(x.loc[:,'%-Anteil an allen Fällen']) for x in merged_data])

fig, axs = plt.subplots(3,4,layout='constrained',figsize=(8,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        im = merged_data[i].plot(ax=ax,column='%-Anteil an allen Fällen', legend=False, cmap=cmap,vmin=vmin,vmax=vmax)
        ax.set(aspect=1.4,title=i+2013)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,cmap,'% of all crime')
fig.suptitle("Germany: Cybercrime per federal state (relative)")
plt.show()

## Fraud
Summenschlüssel:

510000: Betrug §§ 263, 263a, 264, 264a, 265, 265a-e StGB

897100: Computerbetrug § 263a StGB

"Analog" = 51000 - 897100

"Digital" = 897100

In [ ]:
data = LKS01_loader()
merged_data = []
# sumkey 897100 did not exist before 2016 (maybe also an explanation why we see sudden increase in cybercrime for that year)
print('Loading 2016 - 2022 ...')
for year in range(2016,2023):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    df_year_fraud = df_year.loc[df_year['Schlüssel'] == '510000']
    df_year_cfraud = df_year.loc[df_year['Schlüssel'] == '897100']
    df_year_ratio = pd.DataFrame({'Bundesland':df_year_fraud['Bundesland'],
                                  'Anteil':np.array(df_year_cfraud['Anzahl erfasste Fälle']) /
                                  (np.array(df_year_fraud['Anzahl erfasste Fälle']) - np.array(df_year_cfraud['Anzahl erfasste Fälle'])) * 100})
    merged_year = pd.merge(df_year_ratio,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))

In [ ]:
vmin = min([min(x.loc[:,'Anteil']) for x in merged_data])
vmax = max([max(x.loc[:,'Anteil']) for x in merged_data])

fig, axs = plt.subplots(3,3,layout='constrained',figsize=(6,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        merged_data[i].plot(ax=ax,column='Anteil', legend=False, cmap=cmap,vmin=vmin,vmax=vmax)
        ax.set(aspect=1.4,title=i+2016)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,cmap,'% of all fraud')
fig.suptitle("Germany: Computer fraud")
plt.show()


In [ ]:
inhabitants_bu = {'Baden-Württemberg':11124642,
                'Bayern':13176989,
                'Berlin':3677472,
                'Brandenburg':2537868,
                'Bremen':676463,
                'Hamburg':1853935,
                'Hessen':6295017,
                'Mecklenburg-Vorpommern':1611160,
                'Niedersachsen':8027031,
                'Nordrhein-Westfalen':17924591,
                'Rheinland-Pfalz':4106485,
                'Saarland':982348,
                'Sachsen':4043002,
                'Sachsen-Anhalt':2169253,
                'Schleswig-Holstein':2922005,
                'Thüringen':2108863,
                'Bundesrepublik Deutschland':84358845}
inhabs = np.array([11124642,13176989,3677472,2537868,676463,1853935,6295017,1611160,8027031,17924591,4106485,982348,4043002,2169253,2922005,2108863,84358845])


data = LKS01_loader()
merged_data = []
print('Loading 2013 - 2022 ...')
for year in range(2013,2019):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    cases = df_year.loc[df_year['Schlüssel'] == '------']
    merged_year = pd.merge(cases,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))
for year in range(2019,2023):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    cases = df_year.loc[df_year['Schlüssel'] == '------'].copy() # explicitly copying here takes way longer, but gets rid of the warnings...
    cases['HZ nach Zensus'] = cases['Anzahl erfasste Fälle'].to_numpy() / inhabs * 1e5
    merged_year = pd.merge(cases,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))    

In [ ]:
vmin = min([min(x.loc[:,'HZ nach Zensus']) for x in merged_data])
vmax = max([max(x.loc[:,'HZ nach Zensus']) for x in merged_data])

fig, axs = plt.subplots(3,4,layout='constrained',figsize=(8,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        merged_data[i].plot(ax=ax,column='HZ nach Zensus', legend=False, cmap=cmap,vmin=vmin,vmax=vmax)
        ax.set(aspect=1.4,title=i+2013)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,cmap)
fig.suptitle("Germany: Crimes per Capita and Federal State")
plt.show()